In [ ]:
# !pip install ydata-profiling
# !pip install ipywidgets
# !pip install seaborn
# !pip install hvplot
# !pip install ipympl
# !pip install plotly
# !pip install jupyter_bokeh
# ! pip install shap
# !pip install dash dash-bootstrap-components
# !pip install dask
# !pip install dask[distributed]
# !pip install pyarrow
# ! pip install ipywidgets

In [ ]:
%cd ../

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import ipywidgets
sns.set_style("dark")


%matplotlib widget
plt.style.use('https://raw.githubusercontent.com/TDAmeritrade/stumpy/main/docs/stumpy.mplstyle')

In [ ]:
data_path = "data/Quote_and_Order_Data_2025_01_30.csv"

In [ ]:
from gomat_markup_opt.core.train_conversion_probability_model import (
    TOP_10_REGIONS,
    CUSTOMER_COLUMN,
    ORDER_DATE,
    ORDER_ID_COLUMN,
    PRODUCT_ID_COLUMN,
    PLANT_CATEGORY_COLUMN,
    PLANT_NAME_COLUMN,
    QUOTE_STATUS,
    REGION_COLUMN,
    SELLER_PRICE,
    SELLER_PRICE_RELATIVE_TO_MARKET,
    SELLER_TOTAL,
    SELECTED_COLUMNS,
    MARKUP_COLUMN,
    TOTAL_ITEMS_COLUMN,
    REQUEST_PURPOSE_COLUMN,
    QTO_COLUMN,
    BUYER_TOTAL,
)

CONVERSION_MODEL_FEATURE_COLUMNS = [
    PLANT_CATEGORY_COLUMN,
    REGION_COLUMN,
    SELLER_TOTAL,
    MARKUP_COLUMN,
    TOTAL_ITEMS_COLUMN,
    REQUEST_PURPOSE_COLUMN,
    QTO_COLUMN,
    SELLER_PRICE_RELATIVE_TO_MARKET,
]

INFERENCE_COLUMNS = [
    "Request Purpose",
    "qto",
    "Quote/Order Number",
    "Plant Category",
    "product_id",
    "product_size_id",
    "Latin Name",
    "QTY",
    "Seller Price",
    # "Buyer Price",
    "Supplier Name",
    # "Buyer Name",
    "Delivery Address Zip/Postal Code",
    "Buyer Company Region",
    "Supplier_Zip_Postal_Code",
    "Quote/Order Created Date",
]

In [ ]:
PLANT_SPECS_COLUMN = f"{PLANT_NAME_COLUMN}_Specs"
from gomat_markup_opt.core.train_conversion_probability_model import _prepare_input_data

data = pd.read_csv(
    data_path,
    delimiter=",",
    encoding="latin1",
    on_bad_lines="warn",
    engine="python",
)

prepared_data = _prepare_input_data(data)
prepared_data[PLANT_SPECS_COLUMN] = (
    prepared_data[PLANT_NAME_COLUMN] + "_" + prepared_data["Specs"].fillna("")
)
prepared_data = prepared_data.replace({
    QUOTE_STATUS: {
        1:"Converted quote",
        0:"Unconverted quote"
    }
})

In [ ]:
SELLER_PRICE_BINS = "Seller Total Bins"

In [ ]:
import joblib
from gomat_markup_opt.core.train_optimization_space_estimator import (
    MarkupOptSpaceEstimator,
    QuantileBinClassifier,
    Pipeline,
    QuantileLevels,
)

MODELS_FILEPATH = "dev/markup_optimization_interval"

model_path = f"{MODELS_FILEPATH}/estimator.pkl"
interval_estimator = joblib.load(model_path)
interval_estimator

In [ ]:
MODELS_FILEPATH = "dev/model_files2025-03-14_1615"
model_path = f"{MODELS_FILEPATH}/boosting_model.pkl"
conversion_estimator = joblib.load(model_path)
conversion_estimator

In [ ]:
NUM_POINTS = 30
OPTIMAL_MARKUP_COLUMN = "optimal_markup"


def prepare_example(json_example):
    from gomat_markup_opt.core.train_conversion_probability_model import (
        _prepare_input_data,
        _feature_engineering_median_price,
    )

    example_df = pd.DataFrame.from_records([json_example])
    prepared_example = _prepare_input_data(example_df, inference_mode=True)
    prepared_example = _feature_engineering_median_price(prepared_example)

    return prepared_example


def produce_profit_conversion_data(example_df):
    import numpy as np

    prepared_df = interval_estimator.predict(example_df)

    optimization_interval = np.linspace(
        prepared_df["low"].iloc[0], prepared_df["high"].iloc[0], NUM_POINTS
    )

    below_interval = list(np.linspace(10.0, prepared_df["low"].iloc[0], 5))
    above_interval = list(np.linspace(prepared_df["high"].iloc[0], 50, 5))

    results = _get_simulation_results(prepared_df, optimization_interval)
    below_interval_results = _get_simulation_results(prepared_df, below_interval)
    above_interval_results = _get_simulation_results(prepared_df, above_interval)

    return results, below_interval_results, above_interval_results


def _get_simulation_results(prepared_df, optimization_interval):
    results = []
    for markup_to_test in optimization_interval:
        simulated_example = prepared_df.copy(deep=True)
        simulated_example[MARKUP_COLUMN] = markup_to_test
        conversion_prob = conversion_estimator.predict_proba(
            simulated_example[CONVERSION_MODEL_FEATURE_COLUMNS]
        )[0, 1]
        profit = (
            conversion_prob
            * (markup_to_test / 100)
            * simulated_example.iloc[0][SELLER_PRICE]
            * simulated_example.iloc[0][TOTAL_ITEMS_COLUMN]
        )

        results.append(
            {
                "markup": markup_to_test,
                "conversion_chance": conversion_prob,
                "expected_profit": profit,
            }
        )
    return results


def find_best_markups_in_quote(quote_data):
    optimal_markups = []
    for _, line_item in quote_data.iterrows():
        json_example = line_item.to_dict()
        example_df = prepare_example(json_example)
        optimization_results, _, _ = produce_profit_conversion_data(example_df)
        line_item_optimal_markup = max(
            optimization_results, key=lambda x: x["expected_profit"]
        )["markup"]
        optimal_markups.append(line_item_optimal_markup)
    quote_data[OPTIMAL_MARKUP_COLUMN] = optimal_markups
    return quote_data



def main(json_example):
    example_df = prepare_example(json_example)
    optimization_results = produce_profit_conversion_data(example_df)
    return optimization_results

In [ ]:
def get_historical_quotes(item_infos, prepared_data):
    columns = [
        QTO_COLUMN,
        REQUEST_PURPOSE_COLUMN,
        REGION_COLUMN,
        PLANT_NAME_COLUMN,
        "product_size_id",
    ]
    to_retrieve = tuple(item_infos[col] for col in columns)
    try:
        item_historical_quotes = prepared_data.set_index(columns).loc[to_retrieve]
    except KeyError:
        return None
    item_historical_quotes = item_historical_quotes.reset_index()[
        [
            PLANT_NAME_COLUMN,
            SELLER_PRICE,
            MARKUP_COLUMN,
            CUSTOMER_COLUMN,
            ORDER_DATE,
            QUOTE_STATUS,
            REGION_COLUMN,
            "product_size_id",
        ]
    ]
    item_historical_quotes[MARKUP_COLUMN] = item_historical_quotes[MARKUP_COLUMN].round(
        2
    )
    return item_historical_quotes

In [ ]:
def get_ranges(
    df,
    column,
    precision=2,
    num_points=5,
    clip: int | None = None,
    dezoom: float | None = None,
):
    min_y, max_y = df[column].min(), df[column].max() + 1e-1
    if clip is not None:
        max_y = min(clip, max_y)
    if dezoom is not None:
        min_y -= dezoom
        max_y += dezoom
    y_range = np.linspace(min_y, max_y, num_points).round(precision)
    return y_range

In [ ]:
MIN_MARKUP = 10
MAX_MARKUP = 60
X_AXIS_STEPS = 5


def display_results(results_df, b_results_df, a_results_df):
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    # Plot the conversion_chance lineplot on the first axis and add optimization bounds
    x_color = "green"
    sns.lineplot(
        x="markup",
        y="conversion_chance",
        data=results_df,
        ax=axes[0],
        marker="o",
        linewidth=2,
        color="blue",
    )

    sns.lineplot(
        x="markup",
        y="conversion_chance",
        data=b_results_df,
        ax=axes[0],
        linewidth=1,
        color="blue",
    )

    sns.lineplot(
        x="markup",
        y="conversion_chance",
        data=a_results_df,
        ax=axes[0],
        linewidth=1,
        color="blue",
    )

    axes[0].axvline(
        x=results_df["markup"].min(), color=x_color, linestyle="dashed", linewidth=2
    )
    axes[0].axvline(
        x=results_df["markup"].max(), color=x_color, linestyle="dashed", linewidth=2
    )

    axes[0].set_title("Conversion Chance vs. Markup", fontsize=18)
    axes[0].set_xlabel("Markup %", fontsize=14)
    axes[0].set_ylabel("Conversion Chance (0.0 - 1.0)", fontsize=14)
    x0_ranges = np.arange(MIN_MARKUP, MAX_MARKUP, X_AXIS_STEPS)
    y0_ranges = np.arange(0.0, 1.1, 0.2).round(2)
    axes[0].set_xticks(x0_ranges, labels=x0_ranges, fontsize=14)
    axes[0].set_yticks(y0_ranges, labels=y0_ranges, fontsize=14)

    # Plot the profit lineplot on the second axis
    sns.lineplot(
        x="markup",
        y="expected_profit",
        data=results_df,
        ax=axes[1],
        marker="o",
        linewidth=2,
        color="blue",
    )

    sns.lineplot(
        x="markup",
        y="expected_profit",
        data=b_results_df,
        ax=axes[1],
        linewidth=1,
        color="blue",
    )

    sns.lineplot(
        x="markup",
        y="expected_profit",
        data=a_results_df,
        ax=axes[1],
        linewidth=1,
        color="blue",
    )

    axes[1].axvline(
        x=results_df["markup"].min(), color=x_color, linestyle="dashed", linewidth=2
    )
    axes[1].axvline(
        x=results_df["markup"].max(), color=x_color, linestyle="dashed", linewidth=2
    )
    best_markup_idx = results_df["expected_profit"].idxmax()
    axes[1].scatter(
        results_df.iloc[best_markup_idx]["markup"],
        results_df.iloc[best_markup_idx]["expected_profit"],
        s=300,
        c="r",
    )

    axes[1].set_title("Expected Converted Revenue $ vs. Markup %", fontsize=18)
    axes[1].set_xlabel("Markup %", fontsize=14)
    axes[1].set_ylabel("Expected Converted Revenue $", fontsize=14)
    x1_ranges = np.arange(MIN_MARKUP, MAX_MARKUP, X_AXIS_STEPS)
    y1_ranges = get_ranges(
        pd.concat([results_df, b_results_df, a_results_df]), "expected_profit", 0
    )
    axes[1].set_xticks(x1_ranges, labels=x1_ranges, fontsize=14)
    axes[1].set_yticks(y1_ranges, labels=y1_ranges, fontsize=14)

In [ ]:
SELECTED_COLUMNS = [
    ORDER_ID_COLUMN,
    PLANT_NAME_COLUMN,
    "product_size_id",
    "Specs",
    TOTAL_ITEMS_COLUMN,
    SELLER_PRICE,
    MARKUP_COLUMN,
    "Buyer Price",
    SELLER_TOTAL,
    BUYER_TOTAL,
    QUOTE_STATUS,
]

In [ ]:
prepared_data.sort_values(by=ORDER_DATE)[SELECTED_COLUMNS]

In [ ]:
quote_number_widget = ipywidgets.Combobox(
    placeholder="Choose quote number",
    options=prepared_data[ORDER_ID_COLUMN].unique().tolist(),
    description="Quote number",
    ensure_option=True,
    disabled=False,
    style={"description_width": "initial"},
)
quote_number_widget

In [ ]:
SELECTED_QUOTE_NUMBER = quote_number_widget.value

In [ ]:
selected_quote = prepared_data[
    prepared_data[ORDER_ID_COLUMN] == SELECTED_QUOTE_NUMBER
].reset_index()
selected_quote[SELECTED_COLUMNS]

# Markup suggestions

In [ ]:
quote_with_optimal_markup = find_best_markups_in_quote(selected_quote)
quote_with_optimal_markup[SELECTED_COLUMNS + [OPTIMAL_MARKUP_COLUMN]]

# Expected additional revenue

In [ ]:
additional_revenue = (
    (
        (
            quote_with_optimal_markup[OPTIMAL_MARKUP_COLUMN]
            - quote_with_optimal_markup[MARKUP_COLUMN]
        )
        / 100
        * quote_with_optimal_markup[SELLER_PRICE]
        * quote_with_optimal_markup[TOTAL_ITEMS_COLUMN]
    )
    .sum()
    .round()
)
print(f"Additional revenue with optimal markup: ${additional_revenue}")

# How optimal markups are computed

In [ ]:
item_widget = ipywidgets.Select(
    options=sorted(selected_quote[PLANT_SPECS_COLUMN].unique().tolist()),
    description="Line item:",
    disabled=False,
    layout=ipywidgets.Layout(width="50%", height="200px"),
    style={"description_width": "initial"},
)
item_widget

In [ ]:
json_example = (
    selected_quote[selected_quote[PLANT_SPECS_COLUMN] == item_widget.value]
    .iloc[0]
    .to_dict()
)

In [ ]:
json_example

In [ ]:
historical_item_quotes = get_historical_quotes(json_example, prepared_data)
historical_item_quotes = interval_estimator.steps[0][1].transform(
    historical_item_quotes
)
historical_item_quotes = historical_item_quotes[
    historical_item_quotes[SELLER_PRICE_BINS]
    == interval_estimator.predict(prepare_example(json_example))[
        SELLER_PRICE_BINS
    ].iloc[0]
]
historical_item_quotes = historical_item_quotes.sort_values(ORDER_DATE, ascending=False)
historical_item_quotes.head(20)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
ax.hist(historical_item_quotes["Markup"], color="dodgerblue", bins=8)
q25, q75 = np.percentile(historical_item_quotes["Markup"], [10, 90])
x_color = "green"
ax.axvline(q25, color=x_color, linestyle="dashed", linewidth=3)
ax.axvline(q75, color=x_color, linestyle="dashed", linewidth=3)
ax.text(q25, plt.ylim()[1] * 0.9, "10%", color=x_color, fontsize=18)
ax.text(q75, plt.ylim()[1] * 0.9, "90%", color=x_color, fontsize=18)

ax.set_xlabel("Markup %", fontsize=14)
ax.set_ylabel("Count", fontsize=14)

In [ ]:
results, b_results, a_results = main(json_example)
results_df = pd.DataFrame(results)
b_results_df = pd.DataFrame(b_results)
a_results_df = pd.DataFrame(a_results)
display_results(results_df, b_results_df, a_results_df)